# **Base Classifier with 20 image per class** 

In [1]:
import sys

sys.path.insert(0, "..\\Scripts")

In [2]:
from ImageDatastore import ImageDatastore
from torch.utils.data import DataLoader
from torchvision import transforms

## **Dataset**

In [3]:
batch_size = 512
transforms = transforms.Compose(
    [
        transforms.Resize(
            (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
        ),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [4]:
train = ImageDatastore('train_aug', transform=transforms)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

In [5]:
val = ImageDatastore('val_set', transform=transforms)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)

In [6]:
val_deg = ImageDatastore('val_degradate', transform=transforms)
val_deg_loader = DataLoader(val_deg, batch_size=batch_size, shuffle=True)

## **Feature extraction**

In [7]:
from NeuralFeatureExtractor import MobileNetFeatureExtractor

In [8]:
feature_extractor = MobileNetFeatureExtractor()

In [9]:
x_train, y_train = feature_extractor.compute_features(train_loader)
x_train.shape, y_train.shape

0it [00:00, ?it/s]

66it [19:58, 18.16s/it]


((33354, 1280), (33354,))

In [10]:
import os
import numpy as np

In [11]:
base_feature_path = '../Features/features/'
base_labels_path = '../Features/labels/'

In [12]:
np.save(
    os.path.join('..', 'Features', 'features', 'x_train.npy'),
    x_train,
    allow_pickle=False,
)
np.save(
    os.path.join('..', 'Features', 'labels', 'y_train.npy'),
    y_train,
    allow_pickle=False,
)

In [13]:
x_val, y_val = feature_extractor.compute_features(val_loader)
x_val.shape, y_val.shape

24it [06:55, 17.31s/it]


((11994, 1280), (11994,))

In [14]:
np.save(
    os.path.join('..', 'Features', 'features', 'x_val.npy'),
    x_train,
    allow_pickle=False,
)
np.save(
    os.path.join('..', 'Features', 'labels', 'y_val.npy'),
    y_train,
    allow_pickle=False,
)

In [15]:
x_val_deg, y_val_deg = feature_extractor.compute_features(val_deg_loader)
x_val_deg .shape, y_val_deg .shape

24it [06:42, 16.79s/it]


((11994, 1280), (11994,))

In [16]:
np.save(
    os.path.join('..', 'Features', 'features', 'x_val_deg.npy'),
    x_train,
    allow_pickle=False,
)
np.save(
    os.path.join('..', 'Features', 'labels', 'y_val_deg.npy'),
    y_train,
    allow_pickle=False,
)

## **Model**

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score

In [56]:
knn = KNeighborsClassifier(n_neighbors=31, n_jobs=-1, weights='distance', metric='cosine')

In [57]:
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=31,
                     weights='distance')

### **Classification of Test Set**

In [58]:
pred = knn.predict(x_val)
print(accuracy_score(y_val, pred))

0.2467900616975154


In [59]:
pred_score = knn.predict_proba(x_val)
print(top_k_accuracy_score(y_val, pred_score, k=5))

0.4929131232282808


### **Classification of Test Set Degraded**

In [60]:
pred = knn.predict(x_val_deg)
print(accuracy_score(y_val_deg, pred))

0.1878439219609805


In [61]:
pred_score = knn.predict_proba(x_val_deg)
print(top_k_accuracy_score(y_val_deg, pred_score, k=5))

0.39986659996664997
